In [62]:
import distiller 
import numpy as np
import os, collections
import bitstring 
import time 
import torch
import torch.nn as nn
from torch.autograd import Variable
from torchvision import datasets, transforms
import torch.nn.functional as F
import torchvision 
import models 
from matplotlib import pyplot as plt
from eval_util import test_imagenet 
# import multiprocessing 
%matplotlib inline

from fault_injection import * 

import matplotlib 
matplotlib.rcParams['pdf.fonttype'] = 42

print('using GPU:', torch.cuda.is_available())
torch.manual_seed(1)

using GPU: True


In [63]:
# check the weight distribution of other pre-trained models in torch vision
# use resnet18, alexnet, vgg16 

pretrained_models = {'resnet18': torchvision.models.resnet18(pretrained=True),
                     'resnet34': torchvision.models.resnet34(pretrained=True),
                     'alexnet': torchvision.models.alexnet(pretrained=True),
                     'squeezenet': torchvision.models.squeezenet1_0(pretrained=True),
                     'vgg16':  torchvision.models.vgg16(pretrained=True), 
                      'vgg16_bn':  torchvision.models.vgg16_bn(pretrained=True), 
#                      'densenet':  torchvision.models.densenet161(pretrained=True),
                     'inception_v3':  torchvision.models.inception_v3(pretrained=True),
                    }
# for model_name, pretrained_model in pretrained_models.items():
#     tensor = [param.data.cpu().numpy().ravel() for param in pretrained_model.parameters()]
#     tensor = np.concatenate(tensor)
#     minv, maxv = np.min(tensor), np.max(tensor)
#     num_values = tensor.shape[0]
#     plt.hist(tensor, bins=10000)
#     plt.title(model_name+ ':[%f, %f], #=%.1f(M)' %(minv, maxv, num_values/10e6))
#     plt.show()

# model_name = 'vgg16'
# model_name = 'resnet18'
model_name = 'squeezenet'
model = pretrained_models[model_name]
# print(model)

/home/hguan2/anaconda2/envs/distiller/lib/python3.5/site-packages/torchvision/models/squeezenet.py:94: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  init.kaiming_uniform(m.weight.data)
/home/hguan2/anaconda2/envs/distiller/lib/python3.5/site-packages/torchvision/models/squeezenet.py:92: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, mean=0.0, std=0.01)


In [64]:
# check model weight size is a multiple of eight
for name, param in model.named_parameters():
    if len(param.size()) < 2:
        continue 
    length = param.nelement()
    print(name, param.size(), '8X?', length%8 == 0)
    

features.0.weight torch.Size([96, 3, 7, 7]) 8X? True
features.3.squeeze.weight torch.Size([16, 96, 1, 1]) 8X? True
features.3.expand1x1.weight torch.Size([64, 16, 1, 1]) 8X? True
features.3.expand3x3.weight torch.Size([64, 16, 3, 3]) 8X? True
features.4.squeeze.weight torch.Size([16, 128, 1, 1]) 8X? True
features.4.expand1x1.weight torch.Size([64, 16, 1, 1]) 8X? True
features.4.expand3x3.weight torch.Size([64, 16, 3, 3]) 8X? True
features.5.squeeze.weight torch.Size([32, 128, 1, 1]) 8X? True
features.5.expand1x1.weight torch.Size([128, 32, 1, 1]) 8X? True
features.5.expand3x3.weight torch.Size([128, 32, 3, 3]) 8X? True
features.7.squeeze.weight torch.Size([32, 256, 1, 1]) 8X? True
features.7.expand1x1.weight torch.Size([128, 32, 1, 1]) 8X? True
features.7.expand3x3.weight torch.Size([128, 32, 3, 3]) 8X? True
features.8.squeeze.weight torch.Size([48, 256, 1, 1]) 8X? True
features.8.expand1x1.weight torch.Size([192, 48, 1, 1]) 8X? True
features.8.expand3x3.weight torch.Size([192, 48, 3, 

In [65]:
# valdir = '/home/hguan2/datasets/imagenet/val'
# acc1 = test_imagenet(model, valdir, num_batches=50)

In [66]:
# post train quantization 
quantizer = distiller.quantization.PostTrainLinearQuantizer(model)
quantizer.prepare_model()
# print(model)

In [67]:
# acc1 = test_imagenet(model, valdir, num_batches=50)

In [68]:
# use SEC_DED code to encode tensor inplace. use the LSB to record the parity bit
# model.cpu() 
# for name, param in model.named_parameters():
#     if len(param.size()) < 2:
#         continue 
#     tensor = param.data
    
#     start = time.time()
#     secded_encode(tensor)
#     end = time.time() - start
#     print('encode tensor name:%s, size:%s, time(s):%s' %(name, tensor.nelement(), end))
    
# acc1 = test_imagenet(model, valdir, num_batches=50)

In [69]:
def get_named_weights(model):
    named_params = [] 
    for name, param in model.named_parameters():
        if len(param.size()) >= 2:
            named_params.append((name, param)) 
    return named_params 

def large_value_number(tensor):
    size = tensor.nelement()
    num_large_values = torch.nonzero((tensor > 63) + (tensor < -64)).size()[0]
#     print(num_large_values)
    return num_large_values # num_large_values*1.0/size 

# def test_large_value_percentage():
#     tensor = torch.randint(-100, 100, size=(10, ))
#     print(tensor)
#     print(large_value_percentage(tensor))
# test_large_value_percentage()
    
named_params = get_named_weights(model)

n_larges = [] 
weight_id = 0 
for name, param in named_params:
    n_larges.append((weight_id, name, large_value_number(param.data)))
    weight_id += 1
sorted_n_larges = sorted(n_larges, key = lambda x: x[-1], reverse = True)
for item in sorted_n_larges:
    print(item)


(25, 'classifier.1.wrapped_module.weight', 1286)
(23, 'features.12.expand1x1.wrapped_module.weight', 262)
(13, 'features.8.squeeze.wrapped_module.weight', 123)
(0, 'features.0.wrapped_module.weight', 120)
(6, 'features.4.expand3x3.wrapped_module.weight', 117)
(20, 'features.10.expand1x1.wrapped_module.weight', 106)
(11, 'features.7.expand1x1.wrapped_module.weight', 78)
(19, 'features.10.squeeze.wrapped_module.weight', 71)
(4, 'features.4.squeeze.wrapped_module.weight', 69)
(16, 'features.9.squeeze.wrapped_module.weight', 59)
(22, 'features.12.squeeze.wrapped_module.weight', 59)
(2, 'features.3.expand1x1.wrapped_module.weight', 51)
(1, 'features.3.squeeze.wrapped_module.weight', 46)
(18, 'features.9.expand3x3.wrapped_module.weight', 40)
(17, 'features.9.expand1x1.wrapped_module.weight', 39)
(14, 'features.8.expand1x1.wrapped_module.weight', 35)
(8, 'features.5.expand1x1.wrapped_module.weight', 31)
(7, 'features.5.squeeze.wrapped_module.weight', 28)
(12, 'features.7.expand3x3.wrapped_mod

In [70]:
# check the distribution of parameters 
# thr = 32
# layer_id = 0 
# for param_name, param in model.named_parameters():
#     if len(param.size()) < 2:
#         continue
#     counter = collections.Counter(np.abs(param.data.cpu().numpy().ravel())//thr)
#     tmp = sorted(counter.items(), key=lambda x: x[0])
#     values, counts = zip(*tmp)
#     percentages = [count/sum(list(counts)) for count in counts]
#     bar = plt.bar(values, percentages)
#     for rect in bar:
#         height = rect.get_height()
#         plt.text(rect.get_x() + rect.get_width()/2.0, height, '%.4f%%' %(height*100), ha='center', va='bottom')
# #     print(['%.2f' %(p) for p in percentages])
#     #plt.hist(param.data.cpu().numpy().ravel(), bins=10, density=True)
#     plt.xticks(values, [str(int(v)*thr+thr) for v in values])
#     plt.title('layer_id:'+str(layer_id) + ', '+ str(tuple(param.size())))
# #     plt.grid()
#     plt.ylim(0, 1.1)
#     plt.show()
#     layer_id += 1

In [71]:
# check the distribution of parameters all weights
# thr = 32
# total_values, num_weights = 0, 0 
# counter = collections.Counter()
# for param_name, param in model.named_parameters():
#     total_values += param.nelement()
#     if len(param.size()) < 2:
#         continue
#     num_weights += param.nelement()
#     counter.update(collections.Counter(np.abs(param.data.cpu().numpy().ravel())//thr + 1))
    
# tmp = sorted(counter.items(), key=lambda x: x[0])
# values, counts = zip(*tmp)
# total_weights = sum(list(counts))

# assert total_weights == num_weights
# print('#weights:', total_weights, ', #params:', total_values, 'percentage:', '%.6f' %(num_weights/total_values))

# percentages = [count/total_weights for count in counts]
# bar = plt.bar(values, percentages)
# for rect in bar:
#     height = rect.get_height()
#     plt.text(rect.get_x() + rect.get_width()/2.0, height, '%.4f%%' %(height*100), ha='center', va='bottom')
# #     print(['%.2f' %(p) for p in percentages])
# #plt.hist(param.data.cpu().numpy().ravel(), bins=10, density=True)
# # plt.xticks(values, [str(int(v)*thr) for v in values])
# plt.xticks(values, ['[0, 32]', '[32, 64]', '[64, 96]', '[96, 128]'])
# plt.title(model_name)
# #     plt.grid()
# plt.ylim(0, 1.1)
# plt.show()

# figdir = './figures/weight_distribution/'
# figname = model_name+'_int8_weight_distribution.pdf'
# plt.savefig(os.path.join(figdir, figname), bbox_inches='tight')


In [73]:
## load gradual_encoding_absolute result 
lossy_encoding_results = {
    'vgg16': './logs/vgg16/imagenet/int8/gradual_encoding_absolute',
    'resnet18': './logs/resnet18/imagenet/int8/gradual_encoding_absolute',
    'squeezenet': './logs/squeezenet/imagenet/int8/gradual_encoding_absolute',
}


def parse_gradual_encoding_absolute_log(log_path):
    res = [] 
    with open(os.path.join(log_path, 'logs.txt'), 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.strip()
            if line:
                items = line.split(',')
                for item in items:
                    if 'n_large' in item:
                        n_large = int(item.split(':')[-1])
                    if 'name' in item:
                        name = item.split(':')[-1].strip()
                    if 'accuracy' in item:
                        accuracy = float(item.split(':')[-1])
                res.append((accuracy, name, n_large))
                
    return res 

   

log_path = lossy_encoding_results[model_name]
res = parse_gradual_encoding_absolute_log(log_path)

accuracies = [] 
for a, b in zip(sorted_n_larges, res):
    weight_id = a[0]
    accuracy = b[0]
    name = a[1]
    n_large = a[2]
    assert a[1] == b[1], 'param name not equal: %s, %s' %(a[1], b[1])
    assert a[2] == b[2], 'param n_large not equal: %d, %d' %(a[2], b[2])
    accuracies.append((weight_id, name, n_large, accuracy))
for item in accuracies:
    print(item)
            
            


(25, 'classifier.1.wrapped_module.weight', 1286, 67.0)
(23, 'features.12.expand1x1.wrapped_module.weight', 262, 66.86)
(13, 'features.8.squeeze.wrapped_module.weight', 123, 66.64)
(0, 'features.0.wrapped_module.weight', 120, 66.26)
(6, 'features.4.expand3x3.wrapped_module.weight', 117, 66.2)
(20, 'features.10.expand1x1.wrapped_module.weight', 106, 66.42)
(11, 'features.7.expand1x1.wrapped_module.weight', 78, 66.26)
(19, 'features.10.squeeze.wrapped_module.weight', 71, 66.04)
(4, 'features.4.squeeze.wrapped_module.weight', 69, 66.18)
(16, 'features.9.squeeze.wrapped_module.weight', 59, 66.16)
(22, 'features.12.squeeze.wrapped_module.weight', 59, 65.9)
(2, 'features.3.expand1x1.wrapped_module.weight', 51, 66.0)
(1, 'features.3.squeeze.wrapped_module.weight', 46, 65.78)
(18, 'features.9.expand3x3.wrapped_module.weight', 40, 65.86)
(17, 'features.9.expand1x1.wrapped_module.weight', 39, 65.88)
(14, 'features.8.expand1x1.wrapped_module.weight', 35, 65.68)
(8, 'features.5.expand1x1.wrapped_mo

In [75]:
# from right to the left, find the k for sorted_layers[:k].
fault_free_accuracies = {
    
    'vgg16': 79.36,
    'resnet18': 76.46,
    'squeezenet': 66.86,
}
max_acc = fault_free_accuracies[model_name]
min_acc = 0
accuracy_to_ids = [] 
for i in range(len(accuracies)-1, -1, -1):
    accuracy = accuracies[i][-1]
    if accuracy > min_acc:
        accuracy_to_ids.append((accuracy, [x[0] for x in accuracies[:i+1]]))
        min_acc = accuracy 

# add the option of all weights are lossless 
accuracy_to_ids.append((0, []))
    
for item in accuracy_to_ids:
    print(item)

with open(os.path.join(log_path, 'steps.txt'), 'w') as f:
    for item in accuracy_to_ids:
        s = "%.2f" %(item[0]) + ': '+ ', '.join([str(x) for x in item[1]]) + '\n'
        f.write(s) 
    

(64.96, [25, 23, 13, 0, 6, 20, 11, 19, 4, 16, 22, 2, 1, 18, 17, 14, 8, 7, 12, 3, 24, 15, 5, 21, 9, 10])
(65.28, [25, 23, 13, 0, 6, 20, 11, 19, 4, 16, 22, 2, 1, 18, 17, 14, 8, 7, 12, 3, 24, 15, 5, 21])
(65.52, [25, 23, 13, 0, 6, 20, 11, 19, 4, 16, 22, 2, 1, 18, 17, 14, 8, 7, 12, 3, 24, 15])
(65.68, [25, 23, 13, 0, 6, 20, 11, 19, 4, 16, 22, 2, 1, 18, 17, 14, 8, 7, 12])
(65.9, [25, 23, 13, 0, 6, 20, 11, 19, 4, 16, 22, 2, 1, 18, 17, 14, 8, 7])
(66.0, [25, 23, 13, 0, 6, 20, 11, 19, 4, 16, 22, 2])
(66.16, [25, 23, 13, 0, 6, 20, 11, 19, 4, 16])
(66.18, [25, 23, 13, 0, 6, 20, 11, 19, 4])
(66.26, [25, 23, 13, 0, 6, 20, 11])
(66.42, [25, 23, 13, 0, 6, 20])
(66.64, [25, 23, 13])
(66.86, [25, 23])
(67.0, [25])
(0, [])
